<a href="https://colab.research.google.com/github/maao666/Distillation/blob/main/Student_model_gpt2_yuchen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Nov 30 05:23:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.3 MB/s 
     |████████████████████████████████| 7.6 MB 82.0 MB/s 
     |████████████████████████████████| 182 kB 90.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 14.8 MB/s 
     |████████████████████████████████| 132 kB 79.1 MB/s 
     |████████████████████████████████| 212 kB 95.5 MB/s 
     |████████████████████████████████| 127 kB 89.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration,BartModel,DataCollatorForSeq2Seq,TrainingArguments,Seq2SeqTrainer,Seq2SeqTrainingArguments
from transformers import Trainer
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Config
import torch
import csv
from transformers import pipeline


In [3]:
Configuration = GPT2Config.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2',config=Configuration)
device=torch.device('cuda')
model.to(device)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
with open('train_repeat_4000.tsv') as f:
  train = list(csv.reader(f, delimiter="\t"))
# with open('select_data_test.tsv') as f:
#   test = list(csv.reader(f, delimiter="\t"))
# with open('test_repeat_4000.tsv') as f:
#   val = list(csv.reader(f, delimiter="\t"))

In [ ]:
# class DistillDataset(Dataset):
#   def __init__(self,data,tokenizer=tokenizer,max_length=30):
#     self.tokenizer=tokenizer
#     self.input_ids=[]
#     self.attn_masks=[]
#     for i in data:
#       encoding_dict=self.tokenizer('<|startoftext|>'+i[0]+i[1]+'<|endoftext|>',\
#                               truncation=True,max_length=max_length,padding="max_length")
#       self.input_ids.append(torch.tensor(encoding_dict['input_ids']))
#       self.attn_masks.append(torch.tensor(encoding_dict['attention_mask']))
#   def __len__(self):
#     return len(self.input_ids)
#   def __getitem__(self,idx):
#     return self.input_ids[idx],self.attn_masks[idx]

In [ ]:
# dataset_train=DistillDataset(train)
# dataset_val=DistillDataset(val)

In [5]:
def change_form(list_pair):
  my_list=[]
  for pair in list_pair:
    my_list.append({'X':pair[0],'y':pair[1]})
  return my_list

train_change=change_form(train)
# dev_change=change_form(val)
# test_chaneg=change_form(test)

In [ ]:
print(train_change[0]['X'])

PersonX gives PersonY an idea  xAttr  [GEN]


In [6]:
dataset_train = Dataset.from_list(train_change)
# dataset_dev= Dataset.from_list(dev_change)

In [ ]:
# def tokenize_function(sample):
#     input_encodings=tokenizer('<|startoftext|> '+sample['X']+' <|endoftext|>',\
#                               truncation=True,max_length=30,padding="max_length")
#     output_encodings=tokenizer('<|startoftext|> '+sample['y']+' <|endoftext|>',\
#                               truncation=True,max_length=30,padding="max_length")
#     encodings = {
#         'input_ids': input_encodings['input_ids'],
#         'attention_mask': input_encodings['attention_mask'],
#         'labels':output_encodings['input_ids'],
#     }

#     return encodings   


# tokenized_train= dataset_train.map(tokenize_function, batched=False)
# tokenized_dev=dataset_dev.map(tokenize_function, batched=False)

  0%|          | 0/1578153 [00:00<?, ?ex/s]

  0%|          | 0/125636 [00:00<?, ?ex/s]

In [7]:
def tokenize_function(sample):
    input_encodings=tokenizer('<|startoftext|> '+sample['X']+' '+sample['y']+' <|endoftext|>',\
                              truncation=True,max_length=30,padding="max_length")
    # output_encodings=tokenizer('<|startoftext|> '+sample['y']+' <|endoftext|>',\
    #                           truncation=True,max_length=30,padding="max_length")
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels':input_encodings['input_ids'],
    }

    return encodings   


tokenized_train= dataset_train.map(tokenize_function, batched=False)
# tokenized_dev=dataset_dev.map(tokenize_function, batched=False)

  0%|          | 0/2495527 [00:00<?, ?ex/s]

In [8]:
tokenized_train.save_to_disk("tokenized_train")
# tokenized_dev.save_to_disk("tokenized_dev")


# tokenized_train = datasets.load_from_disk("tokenized_train")
# tokenized_dev = datasets.load_from_disk("tokenized_dev")

In [9]:
model.resize_token_embeddings(len(tokenizer))
training_args = TrainingArguments(
    output_dir='test_trainer', 
    evaluation_strategy='epoch',
    per_device_train_batch_size=512, 
    # per_device_eval_batch_size=512, 
    num_train_epochs = 1.0,
    save_strategy="no"
    )

# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    # data_collator=data_collator,
    train_dataset=tokenized_train,
    # eval_dataset=tokenized_train,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

In [10]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: X, y. If X, y are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2495527
  Num Epochs = 1
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 4875
  Number of trainable parameters = 124441344


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


ValueError: ignored

In [21]:
generator = pipeline('text-generation',model=model.cpu(),tokenizer=tokenizer)
# top_p = 0.0
# inp_text=test[0][0]
# inp = tokenizer(inp_text)['input_ids']
# tokens_out = model.generate(torch.tensor(inp).view(1,-1),top_p=top_p, do_sample=top_p > 0.).tolist()[0]
# text_out = tokenizer.decode(tokens_out[len(inp):])
with open('test_repeat_4000.tsv') as f:
  test = list(csv.reader(f, delimiter="\t"))
####
# Print Result
####

# print('INPUT:\n{}\n========'.format(inp_text))
# print('INFERENCE:\n{}\n========'.format(inference))

In [ ]:
# inp_text=test[2][0]
# result=generator(inp_text,num_return_sequences=1)[0]['generated_text'][len(inp_text):]
# print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 to have a business 


In [ ]:
# for line in test:
#   inp_text=line[0]
#   inp = tokenizer(inp_text)['input_ids']
#   tokens_out = model.generate(torch.tensor(inp).view(1,-1).to(device), do_sample = True,  max_length = 30, top_k = 50, top_p = 0.85)
#   text_out = tokenizer.decode(tokens_out[len(inp):])
#   print(text_out)
#   ## trim to sentence end and/or newline
#   if '.' in text_out:
#       text_out = text_out[:text_out.index('.')]
#   if '\n' in text_out:
#       text_out = text_out[:text_out.index('\n')]

#   # finally, trim out only the inference
#   # inference = text_out
#   # print(inference)

In [14]:
from tqdm import tqdm

In [25]:
def output_result(file_name):
  with open(file_name,'w',encoding='utf-8') as f:
    tsv_w = csv.writer(f, delimiter='\t')
    for line in tqdm(test):
      inp_text=line[0]
      text_out=generator(inp_text,num_return_sequences=1)[0]['generated_text'][len(inp_text):]
      inference=text_out.strip()
      tsv_w.writerow([inp_text,inference])  

      # text_out = tokenizer.decode(tokens_out[len(inp):])

      # ## trim to sentence end and/or newline
      # if '.' in text_out:
      #     text_out = text_out[:text_out.index('.')]
      # if '\n' in text_out:
      #     text_out = text_out[:text_out.index('\n')]
      # if '<' in text_out:
      #     text_out = text_out[:text_out.index('<')]
      # # finally, trim out only the inference
      # inference = text_out

In [26]:
output_result("output_4000.tsv")

  0%|          | 0/4000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 5/4000 [00:00<11:59,  5.56it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  1%|          | 32/4000 [00:05<12:01,  5.50it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  1%|          | 39/4000 [00:06<09:05,  7.27it/s]Setting `pad_toke